In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import matplotlib.pyplot as plt
import matplotlib as mpl
params = {
    'axes.labelsize': 8,
    'font.size': 8,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'text.usetex': False,
    'figure.figsize': [4.5, 4.5],
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': True,
}
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams.update(params)
mpl.use("pgf")

import seaborn as sns
sns.set_style("darkgrid")

In [20]:
import torch
import torch.distributions as dist
import torch.nn as nn

cmap = 'YlGnBu'

class BLR(dist.Distribution):
    # https://en.wikipedia.org/wiki/Bayesian_linear_regression
    support = dist.constraints.real_vector
    
    def __init__(self):
        self._event_shape = [2]
        
    def sample(self, sample_shape=torch.Size([])):
        inv_scale = dist.Gamma(1, 1/2).sample(sample_shape)
        scale = inv_scale**-1
        loc = dist.Normal(0, scale.sqrt()).sample()
        scale = (2*torch.bernoulli(0.5*torch.ones(sample_shape))-1)*scale
        return torch.stack([scale, loc], axis=0).T
    
    def log_prob(self, x):
        if x.ndim == 1:
            loc = x[1]
            #scale = x[1].exp()
            #scale = 1 + nn.ELU()(x[1])
            scale = x[0].abs()
        else:
            loc = x[:,1]
            #scale = x[:,1].exp()
            #scale = 1 + nn.ELU()(x[:,1])
            scale = x[:,0].abs()
        return dist.Normal(0, scale.sqrt()).log_prob(loc) + dist.Gamma(1, 1/2).log_prob(scale**-1)

In [35]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

def plot_density(d, ax=None, norm=None, bounds=[-2, 5, -2, 2], exp=True):
    ax.grid(False)
    xmin, xmax, ymin, ymax = bounds
    xx, yy = torch.meshgrid(
        torch.linspace(xmin, xmax, 100),
        torch.linspace(ymin, ymax, 100),
    )
    f = d.log_prob(torch.stack((xx, yy), dim=-1).reshape(-1, 2)).reshape((100,100)).detach().numpy()
    if exp:
        f = np.exp(f)
    if ax is None:
        fig = plt.figure()
        ax = fig.gca()
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    im = ax.imshow(np.rot90(f), cmap=cmap, norm=norm, extent=[xmin, xmax, ymin, ymax], aspect='auto')
    #ax.contour(xx, yy, f, norm=norm, colors='w', linestyles='dashed')
    ax.set_xlabel('$\\sigma$')
    ax.set_ylabel('$\\beta$')
    return ax

# BLR for gaussian (loc) + fat tail (scale) product

In [5]:
# mu = 2*torch.randn(2)
# target = dist.MixtureSameFamily(
#     dist.Categorical(torch.tensor([0.5, 0.5])),
#     dist.MultivariateNormal(
#         torch.stack([mu, -mu]), 
#         torch.stack([torch.eye(2), torch.eye(2)])),
# )
target = BLR()

ax = plot_density(target,
                  )
ax.set_title('Ground Truth')
ax.figure.show()

AttributeError: 'NoneType' object has no attribute 'grid'

In [ ]:
# look at tails along x and y axis
xs = torch.linspace(int(1e4), int(1e6), steps=100)
x_axis = torch.stack([xs, torch.ones_like(xs)]).T

fig, ax = plt.subplots(1,2)
ax[0].plot(xs, target.log_prob(x_axis).detach().numpy())
ax[0].set_title('$\\langle e_0, X-[0;1] \\rangle $')
#ax[0].set_yscale('symlog')

y_axis = torch.stack([torch.ones_like(xs), xs]).T
ax[1].plot(xs, target.log_prob(y_axis).detach().numpy())
ax[1].set_title('$\\langle e_1, X-[0;1] \\rangle $')
#ax[1].set_yscale('symlog')
fig.suptitle('Tail behavior for BLR posterior')

#fig.legend(['vapprox', 'Target'])
fig.show()

In [ ]:
import matplotlib.pyplot as plt

X = BLR().sample((1000,)).numpy()
fig, ax = plt.subplots(1,2)
ax[0].scatter(X[:,0], X[:,1])
ax[1].scatter(X[:,0], X[:,1])
ax[1].set_xlim([-6, 4])
ax[1].set_ylim([-5, 5])
fig.show()

## Approximate it using ADVI

First consider the variational family $q_{\mu \in \mathbb{R}^2,\sigma \in \mathbb{R}^2_{>0}} = \mu + \sigma \odot N(0, I_2)$

## Nflows

In [ ]:
import torch.optim as optim
from tqdm.auto import tqdm
from nflows import transforms, distributions, flows

# Define an invertible transformation.
transform = transforms.CompositeTransform([
    transforms.MaskedAffineAutoregressiveTransform(features=2, hidden_features=256),
    transforms.RandomPermutation(features=2),
    transforms.MaskedAffineAutoregressiveTransform(features=2, hidden_features=256),
    transforms.RandomPermutation(features=2),
    transforms.MaskedAffineAutoregressiveTransform(features=2, hidden_features=256),
    #transforms.MaskedAffineAutoregressiveTransform(features=2, hidden_features=256),
])

# Define a base distribution.
base_distribution = distributions.StandardNormal(shape=[2])


# Combine into a flow.
flow = flows.Flow(transform=transform, distribution=base_distribution)

optimizer = optim.Adam(transform.parameters(), lr=1e-3)

for _ in tqdm(range(1000)):
    optimizer.zero_grad()
    samples = flow.sample(1000)
    #samples = target.sample((1000,))
    log_q = flow.log_prob(samples)
    log_p = target.log_prob(samples)
    loss = (log_q - log_p).mean()
    #loss = -log_q.mean()
    loss.backward()
    optimizer.step()
    tqdm.write(f"Loss: {loss}", end='')


In [ ]:
fig = plot_density(flow,
                   #norm=plt.Normalize(-0.00, 0.0001),
                   bounds=[-6, 4, -5, 5], 
                   norm=plt.Normalize(-0., 0.005)
                   #bounds=[-30, 30, -200, 200]
                  )
#fig = plot_density(vapprox_advi, bounds=[-6, 4, -5, 5])
fig.axes[0].set_title('ADVI')
fig.show()

In [ ]:
# look at tails along x and y axis
xs = torch.linspace(int(1e4), int(1e6), steps=100)
x_axis = torch.stack([xs, torch.ones_like(xs)]).T

fig, ax = plt.subplots(1,2)
ax[0].plot(xs, flow.log_prob(x_axis).detach().numpy())
ax[0].plot(xs, target.log_prob(x_axis).detach().numpy())
ax[0].set_title('$\\langle e_0, X-[0;1] \\rangle $')
#ax[0].set_yscale('symlog')


y_axis = torch.stack([torch.ones_like(xs), xs]).T
ax[1].plot(xs, flow.log_prob(y_axis).detach().numpy())
ax[1].plot(xs, target.log_prob(y_axis).detach().numpy())
ax[1].set_title('$\\langle e_1, X-[0;1] \\rangle $')
#ax[1].set_yscale('symlog')


fig.suptitle('Tail behavior for BLR posterior')
fig.legend(['Flow', 'Target'])
fig.show()


## Use beanmachine

In [ ]:
import beanmachine.ppl as bm
import flowtorch.bijectors
import flowtorch.params
from tqdm.auto import tqdm

@bm.random_variable
def pancake():
    return BLR()

from beanmachine.ppl.experimental.vi.variational_infer import (
    MeanFieldVariationalInference,
)

def on_iter(it, loss, vi_dicts):
    if it % 10 == 0:
        tqdm.write(f"Loss: {loss}", end='')

vi_dicts = MeanFieldVariationalInference().infer(
    queries=[pancake()],
    observations={},
    num_iter=int(1),
    progress_bar=True,
    flow=lambda: flowtorch.bijectors.AffineAutoregressive(
        flowtorch.params.DenseAutoregressive(hidden_dims=(32,32,32))
    ),
#     base_dist=dist.Normal,
#     base_args={
#         'loc': 0.0,
#         'scale': 5.0,
#     },
    lr=1e-2,
    on_iter=on_iter,
    num_elbo_mc_samples=1000,
)

In [ ]:
import torch.optim as optim

target = BLR()
tdist = vi_dicts(pancake()).new_dist
flow_params = vi_dicts(pancake())._flow_params

optimizer = optim.Adam(flow_params.parameters(), lr=1e-3)

for _ in tqdm(range(int(1e3))):
    optimizer.zero_grad()
    samples = tdist.rsample((10000,))
    #samples = target.sample((1000,))
    log_q = tdist.log_prob(samples)
    log_p = target.log_prob(samples)
    loss = (log_q - log_p).mean()
    #loss = -log_q.mean()
    loss.backward(retain_graph=True)
    optimizer.step()
    tqdm.write(f"Loss: {loss}", end='')

In [ ]:
import copy
vapprox_advi = copy.copy(vi_dicts(pancake()))
#estimate_density(vapprox_advi).show()
ax = plot_density(vapprox_advi,
                   #norm=plt.Normalize(-0.01, 0.05),
                  )
#fig = plot_density(vapprox_advi, bounds=[-6, 4, -5, 5])
ax.set_title('ADVI')
ax.figure.show()

In [ ]:
# look at tails along x and y axis
xs = torch.linspace(int(1e4), int(1e5), steps=100)
x_axis = torch.stack([xs, torch.ones_like(xs)]).T

fig, ax = plt.subplots(1,2)
ax[0].plot(xs, vapprox_advi.log_prob(x_axis).detach().numpy())
#ax[0].plot(xs, target.log_prob(x_axis).detach().numpy())
ax[0].set_title('$\\langle e_0, X-[0;1] \\rangle $')
#ax[0].set_yscale('symlog')

y_axis = torch.stack([torch.ones_like(xs), xs]).T
ax[1].plot(xs, vapprox_advi.log_prob(y_axis).detach().numpy())
#ax[1].plot(xs, target.log_prob(y_axis).detach().numpy())
ax[1].set_title('$\\langle e_1, X-[0;1] \\rangle $')
#ax[1].set_yscale('symlog')
fig.suptitle('Tail behavior for BLR posterior')

fig.legend(['vapprox', 'Target'])
fig.show()

In [ ]:
print(f"""
Mean: {vapprox_advi.new_dist.base_dist.base_dist.mean}
Var: {vapprox_advi.new_dist.base_dist.base_dist.variance}
""")

Its variance is higher in the heavy-tailed direction, but it does not capture the tail decay

## Approximate with ATAF

Let us now use ATAF, which uses $q_{\nu \in R_+^2, \mu \in \mathbb{R}^2, \sigma \in R^2_+} = \mu + \sigma \odot StudentT(\nu)$

In [ ]:
import beanmachine.ppl as bm
import flowtorch.bijectors
import torch.nn as nn
from tqdm.auto import tqdm

from beanmachine.ppl.experimental.vi.variational_infer import (
    MeanFieldVariationalInference,
)

def on_iter(it, loss, vi_dicts):
    if it % 10 == 0:
        tqdm.write(f"Loss: {loss}, DF: {vi_dicts(pancake()).new_dist.base_dist.base_dist.df}", end='')

vi_dicts(pancake()).base_dist = lambda **kwargs: dist.Independent(dist.StudentT(**kwargs), 1)
vi_dicts(pancake()).base_arg_constraints = dist.StudentT.arg_constraints
vi_dicts(pancake()).base_args = {
    'df': torch.tensor([int(1e5), int(1e0)]).log(),
    'loc': nn.Parameter(torch.tensor([0.0])),
    'scale': nn.Parameter(torch.tensor([1.0])),
}
vi_dicts(pancake()).recompute_transformed_distribution()

vi_dicts = MeanFieldVariationalInference().infer(
    queries=[pancake()],
    observations={},
    num_iter=int(0),
    progress_bar=True,
    flow=lambda: flowtorch.bijectors.AffineAutoregressive(
        flowtorch.params.DenseAutoregressive(
            hidden_dims=(256,256,256),
        ),
    ),
    #flow=lambda: flowtorch.bijectors.Compose([]),
    vi_dicts=vi_dicts,
    lr=1e-2,
    on_iter=on_iter,
    base_dist=dist.StudentT,
    base_args={
        'df': nn.Parameter(torch.tensor([5.0]).log()),
        'loc': nn.Parameter(torch.tensor([0.0])),
        'scale': nn.Parameter(torch.tensor([1.0])),
    },
    num_elbo_mc_samples=1000,
)

Re-use the good initialization from ADVI

In [ ]:
vi_dicts(pancake()).base_dist = lambda **kwargs: dist.Independent(dist.StudentT(**kwargs), 1)
vi_dicts(pancake()).base_arg_constraints = dist.StudentT.arg_constraints
vi_dicts(pancake()).base_args = {
    'df': torch.tensor([int(2), int(1e10)]).log(),
    'loc': nn.Parameter(torch.tensor([0.0])),
    'scale': nn.Parameter(torch.tensor([1.0])),
}
vi_dicts(pancake()).recompute_transformed_distribution()
vapprox_ftvi = copy.copy(vi_dicts(pancake()))

# look at tails along x and y axis
xs = torch.linspace(int(1e2), int(1e3), steps=100)
x_axis = torch.stack([xs, torch.ones_like(xs)]).T

fig, ax = plt.subplots(1,2)
ax[0].plot(xs, vapprox_ftvi.log_prob(x_axis).detach().numpy())
#ax[0].plot(xs, target.log_prob(x_axis).detach().numpy())
ax[0].set_title('$\\langle e_0, X-[0;1] \\rangle $')
#ax[0].set_yscale('symlog')

y_axis = torch.stack([torch.ones_like(xs), xs]).T
ax[1].plot(xs, vapprox_ftvi.log_prob(y_axis).detach().numpy())
#ax[1].plot(xs, target.log_prob(y_axis).detach().numpy())
ax[1].set_title('$\\langle e_1, X-[0;1] \\rangle $')
#ax[1].set_yscale('symlog')
fig.suptitle('Tail behavior for BLR posterior')

#fig.legend(['vapprox\_ftvi', 'Target'])
fig.show()

In [ ]:
#estimate_density(vapprox_advi).show()
ax = plot_density(vapprox_ftvi, 
                   #norm=plt.Normalize(0, 0.2),
                   #bounds=[-6, 4, -5, 5], 
                   #exp=False,
                  )
ax.set_title('ATAF')
ax.figure.show()

Moreover, it recovers the fat-tailedness and tail-index for the fat-tailed dimension

In [ ]:
print(f"""
Mean: {vapprox_ftvi.new_dist.base_dist.base_dist.mean}
Var: {vapprox_ftvi.new_dist.base_dist.base_dist.variance}
DoF: {vapprox_ftvi.new_dist.base_dist.base_dist.df}
""")

## Approximate with TAF

In [ ]:
import torch.nn as nn

# initialize a MFVApprox
vi_dicts = MeanFieldVariationalInference().infer(
    queries=[pancake()],
    observations={},
    num_iter=1,
    progress_bar=False,
    flow=lambda: flowtorch.bijectors.AffineAutoregressive(
        flowtorch.params.DenseAutoregressive(hidden_dims=(256,256,256))
    ),
    lr=1e-2,
    base_dist=dist.StudentT,
    base_args={
        'df': nn.Parameter(torch.tensor([5.0]).log()),
        'loc': torch.zeros(2),
        'scale': torch.ones(2),
    }
)

# monkey patch it to do TAF
vapprox_taf_fixed = copy.copy(vi_dicts(pancake()))
vapprox_taf_fixed.base_dist = dist.StudentT
vapprox_taf_fixed.base_args = {
    'df': nn.Parameter(vapprox_taf_fixed.base_args['df'].mean()),
    'loc': torch.zeros(2),
    'scale': torch.ones(2).log(),
}
print(vapprox_taf_fixed.recompute_transformed_distribution().df)

Next, we manually write the ELBO training loop (beanmachine doesn't propagate gradients for some reason):

In [ ]:
import torch.optim

optimizer = torch.optim.Adam(vapprox_taf_fixed.parameters(), lr=5e-3)
for _ in tqdm(range(1000)):
    optimizer.zero_grad()
    
    M = 100
    zk = vapprox_taf_fixed.rsample((M,))
    #elbo = Pancake().log_prob(zk).sum()
    elbo = BLR().log_prob(zk).sum()
    elbo -= vapprox_taf_fixed.log_prob(zk).sum()
    elbo /= M
    
    loss = -elbo
    loss.backward(retain_graph=True)
    tqdm.write(f"Loss: {loss.item()}, DF: {vapprox_taf_fixed.new_dist.base_dist.df}", end='')
    #print(vapprox_taf_fixed.base_args['df'])
    optimizer.step()
    vapprox_taf_fixed.recompute_transformed_distribution()
    #print(vapprox_taf_fixed.base_args['df'])

In [ ]:
vapprox_taf_fixed = copy.copy(vi_dicts(pancake()))
vapprox_taf_fixed.base_dist = lambda **kwargs: dist.Independent(dist.StudentT(**kwargs), 1)
vapprox_taf_fixed.base_arg_constraints = dist.StudentT.arg_constraints
vapprox_taf_fixed.base_args = {
    'df': torch.tensor([int(5), int(5)]).log(),
    'loc': nn.Parameter(torch.tensor([0.0])),
    'scale': nn.Parameter(torch.tensor([1.0])),
}
vapprox_taf_fixed.recompute_transformed_distribution()

In [ ]:
# look at tails along x and y axis
xs = torch.linspace(int(1e4), int(1e6), steps=100)
x_axis = torch.stack([xs, torch.ones_like(xs)]).T

fig, ax = plt.subplots(1,2)
ax[0].plot(xs, vapprox_taf_fixed.log_prob(x_axis).detach().numpy())
#ax[0].plot(xs, target.log_prob(x_axis).detach().numpy())
ax[0].set_title('$\\langle e_0, X-[0;1] \\rangle $')
#ax[0].set_yscale('symlog')

y_axis = torch.stack([torch.ones_like(xs), xs]).T
ax[1].plot(xs, vapprox_taf_fixed.log_prob(y_axis).detach().numpy())
#ax[1].plot(xs, target.log_prob(y_axis).detach().numpy())
ax[1].set_title('$\\langle e_1, X-[0;1] \\rangle $')
#ax[1].set_yscale('symlog')
fig.suptitle('Tail behavior for BLR posterior')

#fig.legend(['vapprox\_ftvi', 'Target'])
fig.show()

In [ ]:
ax = plot_density(vapprox_taf_fixed, 
                   #norm=plt.Normalize(0, 0.1),
                   #bounds=[-6, 4, -5, 5], exp=True
                 )
ax.set_title('TAF')
ax.figure.show()

In [ ]:
print(f"""
Mean: {vapprox_taf_fixed.new_dist.base_dist.mean}
Var: {vapprox_taf_fixed.new_dist.base_dist.variance}
DoF: {vapprox_taf_fixed.new_dist.base_dist.base_dist.df}
""")

# Big Plot with Everything

In [22]:
def plot_marginal(axs, d):
    xs = torch.linspace(int(1e4), int(1e6), steps=100)
    x_axis = torch.stack([xs, torch.ones_like(xs)]).T
    y_axis = torch.stack([torch.ones_like(xs), xs]).T
    axs[0].plot(xs, d.log_prob(x_axis).detach().numpy())
    axs[1].plot(xs, d.log_prob(y_axis).detach().numpy())

In [41]:
import beanmachine.ppl as bm
import flowtorch.bijectors
import flowtorch.params
from tqdm.auto import tqdm
import torch.optim as optim

@bm.random_variable
def pancake():
    return BLR()

from beanmachine.ppl.experimental.vi.variational_infer import (
    MeanFieldVariationalInference,
)

def on_iter(it, loss, vi_dicts):
    if it % 10 == 0:
        tqdm.write(f"Loss: {loss}", end='')

vi_dicts = MeanFieldVariationalInference().infer(
    queries=[pancake()],
    observations={},
    num_iter=int(1),
    progress_bar=True,
    flow=lambda: flowtorch.bijectors.AffineAutoregressive(
        flowtorch.params.DenseAutoregressive(hidden_dims=(32,32,32))
    ),
#     base_dist=dist.Normal,
#     base_args={
#         'loc': 0.0,
#         'scale': 5.0,
#     },
    lr=1e-2,
    on_iter=on_iter,
    num_elbo_mc_samples=1000,
)

import torch.optim

target = BLR()
tdist = vi_dicts(pancake()).new_dist
flow_params = vi_dicts(pancake())._flow_params

optimizer = optim.Adam(flow_params.parameters(), lr=1e-3)

for _ in tqdm(range(int(500))):
    optimizer.zero_grad()
    samples = tdist.rsample((1000,))
    #samples = target.sample((1000,))
    log_q = tdist.log_prob(samples)
    log_p = target.log_prob(samples)
    loss = (log_q - log_p).mean()
    #loss = -log_q.mean()
    loss.backward(retain_graph=True)
    optimizer.step()
    tqdm.write(f"Loss: {loss}", end='')

import copy
vapprox_advi = copy.copy(vi_dicts(pancake()))

Loss: tensor([8.7988], grad_fn=<SubBackward0>)


Loss: 0.24911706149578094376


In [42]:
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy

cols = [
    '$p(\\beta, \\sigma)$',
    '$p(\\beta=1, \\sigma)$',
    '$p(\\beta, \\sigma=1)$',
]
rows = [
    'Target',
    'Gaussian',
    'TAF',
    'ATAF',
]

fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(5.5, 4.5))
#plt.setp(axes.flat, xlabel='X-label', ylabel='Y-label')

pad = 5 # in points

for row, d in enumerate([
    target, vi_dicts(pancake()), vi_dicts(pancake()), vi_dicts(pancake()),
]):
    if row == 0:
        plot_density(d, axes[row][0],
                     norm=plt.Normalize(0.07, 0.13),
                     bounds=[1,8,-2,2],
                    )
        plot_marginal(axes[row][1:3], d)
    else:
        plot_density(d, axes[row][0], 
                     bounds=[1,8,-2,2],
                     #norm=plt.Normalize(-0.00, 0.5)
                    )
        
        if row > 1:
            if row == 2:
                dfs = torch.tensor([int(5), int(5)]).log()
            elif row == 3:
                dfs = torch.tensor([int(1e12), int(1e0)]).log()

            d.base_dist = lambda **kwargs: dist.Independent(dist.StudentT(**kwargs), 1)
            d.base_arg_constraints = dist.StudentT.arg_constraints
            d.base_args = {
                'df': dfs,
                'loc': nn.Parameter(torch.tensor([0.0])),
                'scale': nn.Parameter(torch.tensor([1.0])),
            }
            d.recompute_transformed_distribution()
        plot_marginal(axes[row][2:0:-1], d)
        
    axs =  axes[row][1:3]
    axs[0].set_xlabel('$\\sigma$')
    axs[1].set_xlabel('$\\beta$')
        


    


for ax, col in zip(axes[0], cols):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size='large', ha='center', va='baseline')

for ax, row in zip(axes[:,0], rows):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center', rotation=90)

fig.tight_layout(pad=0, w_pad=0, h_pad=0)
# tight_layout doesn't take these labels into account. We'll need 
# to make some room. These numbers are are manually tweaked. 
# You could automatically calculate them, but it's a pain.
fig.subplots_adjust(top=0.9)

fig.show()

In [43]:
fig.savefig('blr_aniso.pdf')